In [15]:
# test_dataset.head()
test_dataset["A1-rank"]

0         x
1         x
2         x
3        a+
4        a+
       ... 
2073      x
2074      s
2075    NaN
2076      x
2077     s+
Name: A1-rank, Length: 2078, dtype: object

In [1]:
# imports
import numpy as np  # Required version 1.19.2
import pandas as pd # Required version 1.1.3
import matplotlib.pyplot as plt
from nbc import NBC

In [2]:
"""
Predict win based on team and opponent team weapon and level
"""

# Read in dataset
dataset_dir = "./dataset/"
train_dataset_path = dataset_dir + "2020-11-19.csv"
test_dataset_path = dataset_dir + "2020-11-20.csv"


used_columns = [
    "lobby-mode",
    "lobby",
    "mode",
    "stage",
    "win",

    "A1-weapon",
    "A1-rank",
    "A1-level",
    "A2-weapon",
    "A2-rank",
    "A2-level",
    "A3-weapon",
    "A3-rank",
    "A3-level",
    "A4-weapon",
    "A4-rank",
    "A4-level",

    "B1-weapon",
    "B1-rank",
    "B1-level",
    "B2-weapon",
    "B2-rank",
    "B2-level",
    "B3-weapon",
    "B3-rank",
    "B3-level",
    "B4-weapon",
    "B4-rank",
    "B4-level"
]

train_dataset = pd.read_csv(train_dataset_path, engine="python", usecols=used_columns)

# Drop NA (Use Ranked and League battle data only for now)
train_dataset = train_dataset.dropna()
label_name = "win"


In [3]:
train_dataset.head()

,lobby-mode,lobby,mode,stage,win,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,...,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,gachi,squad_4,hoko,bbass,alpha,kelvin525_becchu,x,166,dynamo,x,...,302,splatroller,b-,76,bamboo14mk1,s+,156,dualsweeper_custom,x,95.0
1,gachi,squad_4,hoko,bbass,alpha,kelvin525_becchu,x,166,dynamo,x,...,380,dualsweeper_custom,x,243,herocharger_replica,x,331,screwslosher_becchu,s+,198.0
2,gachi,squad_4,hoko,fujitsubo,alpha,kelvin525_becchu,x,166,dynamo,x,...,447,nzap85,s+,198,rapid_deco,x,449,parashelter_sorella,x,359.0
3,gachi,standard,yagura,otoro,alpha,bold_7,a+,299,jetsweeper_custom,a-,...,71,bold,a,136,promodeler_mg,a,58,herocharger_replica,a-,109.0
4,gachi,standard,yagura,otoro,bravo,bold_7,a+,299,nzap89,a-,...,135,jetsweeper,a+,138,nova,a,79,dualsweeper_custom,a,282.0


In [4]:
train_dataset.dtypes

lobby-mode     object
lobby          object
mode           object
stage          object
win            object
A1-weapon      object
A1-rank        object
A1-level        int64
A2-weapon      object
A2-rank        object
A2-level        int64
A3-weapon      object
A3-rank        object
A3-level        int64
A4-weapon      object
A4-rank        object
A4-level      float64
B1-weapon      object
B1-rank        object
B1-level        int64
B2-weapon      object
B2-rank        object
B2-level        int64
B3-weapon      object
B3-rank        object
B3-level        int64
B4-weapon      object
B4-rank        object
B4-level      float64
dtype: object

In [5]:
# Model config
stages = [
    'anchovy',
    'arowana',
    'bbass',
    'mongara',
    'ajifry',
    'kombu',
    'ama',
    'mozuku',
    'zatou',
    'manta',
    'tachiuo',
    'fujitsubo',
    'otoro',
    'shottsuru',
    'hokke',
    'devon',
    'mutsugoro',
    'engawa',
    'gangaze',
    'chozame',
    'battera',
    'sumeshi',
    'hakofugu',
    'mystery_11',
    'mystery_21',
    'mystery_07',
    'mystery_05',
    'mystery_18',
    'mystery_23',
    'mystery_03',
    'mystery_09',
    'mystery_16',
    'mystery_24',
    'mystery_15',
    'mystery',
    'mystery_12',
    'mystery_14',
    'mystery_08',
    'mystery_22',
    'mystery_20',
    'mystery_17',
    'mystery_02',
    'mystery_19',
    'mystery_13',
    'mystery_01',
    'mystery_04',
    'mystery_10',
    'mystery_06'
]

weapons = [
    '52gal',
    '52gal_becchu',
    '52gal_deco',
    '96gal',
    '96gal_deco',
    'bold',
    'bold_7',
    'bold_neo',
    'bottlegeyser',
    'bottlegeyser_foil',
    'heroshooter_replica',
    'jetsweeper',
    'jetsweeper_custom',
    'momiji',
    'nzap83',
    'nzap85',
    'nzap89',
    'ochiba',
    'octoshooter_replica',
    'prime',
    'prime_becchu',
    'prime_collabo',
    'promodeler_mg',
    'promodeler_pg',
    'promodeler_rg',
    'sharp',
    'sharp_neo',
    'sshooter',
    'sshooter_becchu',
    'sshooter_collabo',
    'wakaba',
    'clashblaster',
    'clashblaster_neo',
    'heroblaster_replica',
    'hotblaster',
    'hotblaster_custom',
    'longblaster',
    'longblaster_custom',
    'longblaster_necro',
    'nova',
    'nova_becchu',
    'nova_neo',
    'rapid',
    'rapid_becchu',
    'rapid_deco',
    'rapid_elite',
    'rapid_elite_deco',
    'h3reelgun',
    'h3reelgun_cherry',
    'h3reelgun_d',
    'l3reelgun',
    'l3reelgun_becchu',
    'l3reelgun_d',
    'dualsweeper',
    'dualsweeper_custom',
    'heromaneuver_replica',
    'kelvin525',
    'kelvin525_becchu',
    'kelvin525_deco',
    'maneuver',
    'maneuver_becchu',
    'maneuver_collabo',
    'quadhopper_black',
    'quadhopper_white',
    'sputtery',
    'sputtery_clear',
    'sputtery_hue',
    'carbon',
    'carbon_deco',
    'dynamo',
    'dynamo_becchu',
    'dynamo_tesla',
    'heroroller_replica',
    'splatroller',
    'splatroller_becchu',
    'splatroller_collabo',
    'variableroller',
    'variableroller_foil',
    'herobrush_replica',
    'hokusai',
    'hokusai_becchu',
    'hokusai_hue',
    'pablo',
    'pablo_hue',
    'pablo_permanent',
    'bamboo14mk1',
    'bamboo14mk2',
    'bamboo14mk3',
    'herocharger_replica',
    'liter4k',
    'liter4k_custom',
    'liter4k_scope',
    'liter4k_scope_custom',
    'soytuber',
    'soytuber_custom',
    'splatcharger',
    'splatcharger_becchu',
    'splatcharger_collabo',
    'splatscope',
    'splatscope_becchu',
    'splatscope_collabo',
    'squiclean_a',
    'squiclean_b',
    'squiclean_g',
    'bucketslosher',
    'bucketslosher_deco',
    'bucketslosher_soda',
    'explosher',
    'explosher_custom',
    'furo',
    'furo_deco',
    'heroslosher_replica',
    'hissen',
    'hissen_hue',
    'screwslosher',
    'screwslosher_becchu',
    'screwslosher_neo',
    'barrelspinner',
    'barrelspinner_deco',
    'barrelspinner_remix',
    'herospinner_replica',
    'hydra',
    'hydra_custom',
    'kugelschreiber',
    'kugelschreiber_hue',
    'nautilus47',
    'nautilus79',
    'splatspinner',
    'splatspinner_becchu',
    'splatspinner_collabo',
    'campingshelter',
    'campingshelter_camo',
    'campingshelter_sorella',
    'heroshelter_replica',
    'parashelter',
    'parashelter_sorella',
    'spygadget',
    'spygadget_becchu',
    'spygadget_sorella'
 ]

ranks = [
    "c-",
    "c",
    "c+",
    "b-",
    "b",
    "b+",
    "a-",
    "a",
    "a+",
    "s",
    "s+",
    "x"
]

In [7]:
model_params = {
    "smoothing_params": {
        "lobby-mode": ["gachi", "regular", "fest"],
        "lobby": ["standard", "squad_2", "squad_4", "private"],
        "mode": ["nawabari", "area", "yagura", "hoko", "asari"],
        "stage": stages,
        "A1-weapon": weapons,
        "A1-rank": ranks,
        "A2-weapon": weapons,
        "A2-rank": ranks,
        "A3-weapon": weapons,
        "A3-rank": ranks,
        "A4-weapon": weapons,
        "A4-rank": ranks,
        "B1-weapon": weapons,
        "B1-rank": ranks,
        "B2-weapon": weapons,
        "B2-rank": ranks,
        "B3-weapon": weapons,
        "B3-rank": ranks,
        "B4-weapon": weapons,
        "B4-rank": ranks
    },
    "continuous": {
        "A1-level": True,
        "A2-level": True,
        "A3-level": True,
        "A4-level": True,
        "B1-level": True,
        "B2-level": True,
        "B3-level": True,
        "B4-level": True
    }
}

In [8]:
# Create model
nbc = NBC(model_params)

# Train on model
nbc.train(train_dataset, label_name=label_name, partition="qcut", q=5, smoothing=False)


{'alpha': 0.5560453400503779, 'bravo': 0.4439546599496222}


In [16]:
# Load Test dataset
test_dataset = pd.read_csv(test_dataset_path, engine="python", usecols=used_columns)

# Drop NA (Use Ranked and League battle data only for now)
test_dataset = test_dataset.dropna()
label_name = "win"

In [17]:
test_dataset.head()

,lobby-mode,lobby,mode,stage,win,A1-weapon,A1-rank,A1-level,A2-weapon,A2-rank,...,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,gachi,squad_4,hoko,bbass,alpha,kelvin525_becchu,x,166,dynamo,x,...,302,splatroller,b-,76,bamboo14mk1,s+,156,dualsweeper_custom,x,95.0
1,gachi,squad_4,hoko,bbass,alpha,kelvin525_becchu,x,166,dynamo,x,...,380,dualsweeper_custom,x,243,herocharger_replica,x,331,screwslosher_becchu,s+,198.0
2,gachi,squad_4,hoko,fujitsubo,alpha,kelvin525_becchu,x,166,dynamo,x,...,447,nzap85,s+,198,rapid_deco,x,449,parashelter_sorella,x,359.0
3,gachi,standard,yagura,otoro,alpha,bold_7,a+,299,jetsweeper_custom,a-,...,71,bold,a,136,promodeler_mg,a,58,herocharger_replica,a-,109.0
4,gachi,standard,yagura,otoro,bravo,bold_7,a+,299,nzap89,a-,...,135,jetsweeper,a+,138,nova,a,79,dualsweeper_custom,a,282.0


In [10]:
# Evaluate model
nbc.evaluate(test_dataset, label_name)

KeyError: 'regular'